# Programmation d'un régulateur numérique

## Généralités

Un régulateur numérique est généralement décrit par sa fonction de transfert en z. Partons, en toute généralité, de la fonction de transfert discrète suivante:

$$ K(z) = \frac{U(z)}{E(z)} = \frac{b_0 z^n + b_1 z^{n-1} + \dots + b_n}{z^n + a_1 z^{n-1} + \dots + a_n} $$

Sous cette forme, la fonction de transfert est utile pour la synthèse du régulateur, mais pas directement pour la programmation de l'algorithme de commande. Il est plus utile de passer en puissances négatives de z:

$$ K(z^{-1}) = \frac{b_0 + b_1 z^{-1} + \dots + b_n z^{-n}}{1 + a_1 z^{-1} + \dots + a_n z^{-n}} $$

Il est maintenant facile de convertir la fonction de transfert en algorithme temporel:

\begin{align} 
  U(z) \left( 1 + a_1 z^{-1} + \dots + a_n z^{-n} \right) &= E(z) \left( b_0 + b_1 z^{-1} + \dots + b_n z^{-n} \right) \\
  U(z) + a_1 z^{-1} U(z) + \dots + a_n z^{-n} U(z) &= b_0 E(z) + b_1 z^{-1} E(z) + \dots + b_n z^{-n} E(z)
\end{align}

En se souvenant que $z^{-1}$ représente un retard d'une période d'échantillonnage, on obtient:

\begin{align}
  & u(k) + a_1 u(k-1) + \dots + a_n u(k-n) = b_0 e(k) + b_1 e(k-1) + \dots + b_n e(k-n) \\
  & u(k) = -a_1 u(k-1) - \dots - a_n u(k-n) + b_0 e(k) + b_1 e(k-1) + \dots + b_n e(k-n)
\end{align}  

La forme développée ici, générant une commande à l'instant $t_k$ dépendant de l'écart au même instant, est appelée ***implémentation standard***. Cette forme suppose donc que l'algorithme de commande ne contient pas de retard pur.

En pratique, par exemple lors de l'utilisation d'un automate programmable (**PLC** ou **PAC**), retard existe entre l'évaluation de l'erreur à l'instant $t_k$ et la mise à jour de la commande. Ce retard, en fonction de la période fixée, peut être soit négligeable par rapport à la période d'échantillonnage, soit proche de la période.

Il peut aussi arriver d'imposer au processeur de mettre à jour la commande en fin de période, afin de s'assurer d'avoir toujours le même retard entre l'évaluation de l'erreur et la mise à jour de la commande.

Dans ce dernier cas, on ajoute un retard d'une période d'échantillonnage à la fonction de transfert du régulateur, ce qui donne l'algorithme temporel suivant:

$$ u(k) = -a_1 u(k-1) - \dots - a_n u(k-n) + b_0 e(k-1) + b_1 e(k-1) + \dots + b_n e(k-n) $$

## Algorithme de commande

Pour programmer le régulateur, on utilisera l'algorithme suivant:

- lors du tick d'horloge, à l'instant $t_k$, la sortie $y(k)$ est mesurée
- l'erreur $e(k) = y_c(k) - y(k)$ est alors calculée; la consigne est en général stockée en mémoire 
- la commande est évaluée grâce à l'algorithme temporel:
$$ u(k) = -a_1 u(k-1) - \dots - a_n u(k-n) + b_0 e(k-1) + b_1 e(k-1) + \dots + b_n e(k-n) $$
où les coefficients $a_i,\, b_i\: i = 0, 1, \dots, n$ sont tirés de la mémoire
- la conversion digital-analogique de $u(k)$ peut être effectuée directement après son calcul afin de diminuer le temps de retard
-  enfin, on met à jour toutes les variables $u(k-1), u(k-2), \cdots, u(k-n), e(k-1), e(k-2), \cdots, e(k-n)$ par $u(k), u(k-1), \cdots, u(k-n+1), e(k), e(k-1), \cdots, e(k-n+1)$; cette mise à jour est évidemment effectuée à rebours

## Cas du régulateur PID

Pour rappel, la seconde méthode d'Euler donne la fonction de transfert du PID mixte suivante:

$$ K(z) = K_p \left( 1 + \frac{\frac{h}{T_i}z}{z-1} + \frac{N(z-1)}{\left( 1 + N\frac{h}{T_d} \right) z - 1} \right) $$

Au lieu d'utiliser l'algorithme général proposé à la section précédente, il est préférable de considérer les 3 actions séparément. Cette astuce permettra de faciliter la gestion du terme intégral, lors de l'ajout des mesures ARW.

On pose:

\begin{align}
  U_i(z) &= \frac{\frac{h}{T_i}z}{z-1} E(z) \\
  U_d(z) &= \frac{N(z-1)}{\left( 1 + N\frac{h}{T_d} \right) z - 1} E(z)
\end{align}

En passant par les puissances négatives de z:

\begin{align}
  U_i(z) &= \frac{\frac{h}{T_i}}{1-z^{-1}} E(z) \\
  U_d(z) &= \frac{N(1-z^{-1})}{\left( 1 + N\frac{h}{T_d} \right) - z^{-1}} E(z)
\end{align}

Il en résulte l'algorithme suivant:

\begin{align}
  & e(kh) = y_c(kh) - y(kh) \\
  & u_i(kh) = u_i(kh-h) + \frac{h}{T_i} e(kh) \\
  & u_d(kh) = \frac{T_d}{T_d + Nh} \left( u_d(kh-h) + N \left( e(kh) - e(kh-h) \right) \right) \\
  & u(kh) = K_p (e(kh) + u_i(kh) + u_d(kh))
\end{align}

### Erreur d'intégration

Un souci peut cependant arriver avec l'algorithme présenté ci-dessus. Dans le cas où la période d'échantillonnage est petite, le terme $e(kh) - e(kh-h)$ sera très petit. Les erreurs d'arrondi inhérentes au calcul numérique deviennent alors importantes.

De plus, le terme $\frac{h}{T_i} e(kh)$, lorsque la période d'échantillonnage est petite et que $T_i$ est grand, devient tellement petit qu'il est considéré par le processeur comme égal à 0. L'intégral n'a donc plus aucun effet alors que l'écart n'est toujours pas nul. Ce phénomène est appelé ***erreur d'intégration***.

Une dernière remarque peut être faite par rapport à la période d'échantillonnage. Lorsqu'elle est trop petite, pour des besoins de visualisation et d'historisation des données, il faudra prévoir une plus grosse mémoire afin de stocker toutes ces données, ainsi qu'une bande bassante importante afin de transmettre ces données vers le serveur de stockage. Sans compter aussi le prix du matériel d'acquisition de données qui augmente avec la fréquence d'échantillonnage. Il vaut donc mieux ne pas inutilement diminuer la période d'échantillonnage.

Afin de résoudre le problème à l'origine de l'erreur d'intégration, l'intégrale peut être modifiée ainsi:

\begin{align}
  i(kh) &= i(kh-h) + e(kh) \\
  u_i(kh) &= \frac{h}{T_i} i(kh)
\end{align}

Cette solution permet de multiplier le terme $h/T_i$ par $i(kh)$, représentant la somme des erreurs depuis le lancement de la commande. Lorsque l'erreur diminue, le terme $i(kh)$ continue d'augmenter. Il en résulte que, même si à certains instants, la commande $u_i(kh)$ ne varie pas, elle continuera d'évoluer aux instants suivants.

Il faudra par contre stocker la variable $i(kh)$ sur un mot suffisamment long car la somme pourrait amener à un dépassement de capacité.

### Mesure anti-emballement

Pour rappel, la seconde méthode anti-emballement gèle l'intégration lorsque $|u(kh)| > \mu$:

$$ u_i(kh) = u_i(kh-h) $$

Il suffit donc de comparer $u(kh)$ à la saturation et, dans le cas où l'inégalité précédente est respectée, ajouter la ligne précédente.

### Algorithme complet

En tenant compte de toutes les mesures précédentes, l'algorithme devient:

$$
\begin{align}
  & e(kh) = y_c(kh) - y(kh) \\
  & i(kh) = i(kh-h) + e(kh) \\
  & u_i(kh) = \frac{h}{T_i} i(kh) \\
  & u_d(kh) = \frac{T_d}{T_d + Nh} \left( u_d(kh-h) + N \left( e(kh) - e(kh-h) \right) \right) \\
  & u(kh) = K_p (e(kh) + u_i(kh) + u_d(kh)) \\
  & \text{si}\: |u(kh)| > \mu \: \text{alors}: \\
  &  \qquad u_i(kh) = u_i(kh-h) \\
  &  \qquad u(kh) = K_p (e(kh) + u_i(kh) + u_d(kh))
\end{align}
$$